In [2]:
import requests
import pandas as pd
import numpy as np
import json
# Progress bars
import tqdm
from tqdm import tqdm_notebook
import lzma
from multiprocessing import Pool
import os

from collections import OrderedDict, defaultdict
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    #'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

In [3]:
with open('../v3_json_data/genrebiglabels.json') as f:
    genres_list = json.load(f)

# Genres and starting points:
genres = [(feature['properties']['name'], feature['geometry']['coordinates']) for feature in genres_list['features']]

In [4]:
genres[4][0].lower().replace(' ', '')

'dirtysouthrap'

In [77]:
#genres

In [6]:
# Tracklists: http://music.ishkur.com/musicbox.php?x=9540&y=7685&match=genre
# Descriptions: http://music.ishkur.com/content.php?g=Microfunk

In [8]:
def save_genre(params):
    
    name = params[0].replace(' ', '').replace('/','%2F')
    
    desc_name = params[0].replace(' ','%20').replace('/','%2F').replace('&', '%26amp%3B')
    
    x = params[1][0]
    y = params[1][1]
    
    desc_fname = "../v3_html_data/{}_description.html.xz".format(name.lower())
    desc_url = "http://music.ishkur.com/content.php?g={}".format(desc_name)
    cache_url(desc_url, desc_fname) #
    
    tracks_fname = "../v3_html_data/{}_tracks.html.xz".format(name.lower())
    tracks_url = "http://music.ishkur.com/musicbox.php?x={}&y={}&match=genre".format(x,y)
    cache_url(tracks_url, tracks_fname)

# Save a given url as a file
def cache_url(url, fname, overwrite=False):
    if os.path.isfile(fname) and not overwrite:
        return

    try: 
        data = requests.get(url, stream=False, timeout=30, allow_redirects=True, headers=headers).content
    except Exception as e:
        print("error:", url)
        return
    
    with lzma.open(fname, 'wb') as f:            
        f.write(data)

# Show Progress bar and perform a function on an iterable in parallel:
def process_with_progress(func, iterable):
    with Pool(8) as p:
        results = list(tqdm_notebook(p.imap(func, iterable), total=len(iterable)))
    return results

In [9]:
save_genre(genres[0])

In [10]:
process_with_progress(save_genre, genres)
print()

In [11]:
# Derive Scene based on position:
#def y_sort(item):
#    return item['geometry']['coordinates'][1]

#with open('../v3_json_data/scenelabels.json') as f:
#    scenes = json.load(f)

#iter_scenes = iter(sorted(scenes['features'], key=y_sort))#
#
#first = next(iter_scenes)
#y = first['geometry']['coordinates'][1]
#name = first['properties']['name']#
#
#scene_dict = {}
#for scene in iter_scenes:#
#
#    next_y = scene['geometry']['coordinates'][1]
#    next_name = scene['properties']['name']
#    
#    scene_dict[range(y, next_y)] = name#
#
#    y = next_y
#    name = next_name#
#
#scene_dict

In [12]:
#with open('../v3_json_data/genrebiglabels.json') as f:
#    genres = json.load(f)
#
#_ = [print(json.dumps(s)) for s in sorted(genres['features'], key=y_sort)]
# Manually split - genre big label y coords not helpful, ovelap with labels

In [13]:
#sorted(scenes['features'], key=y_sort)

In [14]:
#[scene_dict[k] for k in scene_dict.keys() if 1305 in k]

In [15]:
#{ "type": "Feature", "properties": { "name": "Trap"}, "geometry": { "type": "Point", "coordinates": [19430, 18605] } }, #bass
#{ "type": "Feature", "properties": { "name": "Southern Rap"}, "geometry": { "type": "Point", "coordinates": [12530, 18505] } }, #hip hop

In [16]:
def y_sort(item):
    return item['geometry']['coordinates'][1]

with open('../v3_json_data/sorted_genres.jsonl') as f:
    # Manually split by newlines
    groups_lines = f.read().split('\n\n')    
    groups = [[json.loads(g) for g in group.split('\n') if g is not ''] for group in groups_lines]
    print(len(groups))
    
with open('../v3_json_data/scenelabels.json') as f:
    scenes = sorted(json.load(f)['features'], key=y_sort)
    scenes = [scene['properties']['name'] for scene in scenes]
    # instert extra pioneers to cover minimalism etc. (spans across IDM)
    scenes.remove('Pioneers')
    scenes.insert(9, 'Pioneers')
    scenes.insert(10, 'Intelligent Dance Music')
    scenes.insert(12,'Pioneers')
    scenes.insert(15,'Pioneers')
    print(len(scenes))

31
31


In [17]:
groups[8]

[{'type': 'Feature',
  'properties': {'name': 'Breakcore'},
  'geometry': {'type': 'Point', 'coordinates': [17030, 5605]}},
 {'type': 'Feature',
  'properties': {'name': 'Drill n Bass'},
  'geometry': {'type': 'Point', 'coordinates': [14330, 5705]}},
 {'type': 'Feature',
  'properties': {'name': 'Glitch Hop'},
  'geometry': {'type': 'Point', 'coordinates': [17030, 5805]}},
 {'type': 'Feature',
  'properties': {'name': 'Glitch'},
  'geometry': {'type': 'Point', 'coordinates': [16430, 5905]}}]

In [18]:
scenes[8]

'Intelligent Dance Music'

In [76]:
# oops. overwrites, multiple entries..
#scene_to_genre = OrderedDict(zip(scenes, [[g['properties']['name'] for g in group] for group in groups]))

scene_to_genre = OrderedDict()
for scene,group_names in zip(scenes, [[g['properties']['name'] for g in group] for group in groups]):
    #print (scene, group_names)
    if scene not in scene_to_genre:
        scene_to_genre[scene] = group_names
    else:
        g_list = scene_to_genre[scene]
        g_list.extend(group_names)
        scene_to_genre[scene] = g_list

genre_to_scene = OrderedDict((i,k) for k,v in scene_to_genre.items() for i in v)
#genre_to_scene

In [20]:
#scene_to_genre['Industrial/Goth']

In [21]:
#[[(k,i) for i in v] for k,v in scene_to_genre.items()]

In [22]:
#with open('../v3_json_data/coords.json') as f:
#    coords = json.load(f)['features']

In [23]:
#coords[140]['geometry']['coordinates'][0]

In [24]:
#coords[164]

In [25]:
# Coords for years: 2018 start: 21900, ever 300 is a year.
# coords from gcpoly: [[ [22200, 6125], [22200, 6085], [21900, 6085], [21900, 6125], [22200, 6125] ]]
                    #       right  Top    right bottom  left   bottom   left top       right top

In [26]:
# typo: eastcost / eastcoast rap; LIquid / Liquid funk; US Deep House (1992) Mr. Fingers'; 

with open('../v3_json_data/gcpoly.json') as f:
    coords = json.load(f)['features']

In [27]:
# x values to years:
x_to_year = dict(zip(range(22200, 0, -300), range(2019,1946,-1)))

In [28]:
def extract(coord):
    # Index into left coord x
    query_point = sorted(coord['geometry']['coordinates'][0])[0]
    return {"name":coord['properties']['name'], "tracks": coord['properties']['value'], "year":x_to_year[query_point[0]], "query": query_point}


In [29]:
all_years = [extract(coord) for coord in coords] 

In [30]:
len(all_years)

2856

In [31]:
all_years[2]

{'name': '2-Step Garage', 'tracks': '12', 'year': 2000, 'query': [16500, 9485]}

In [32]:
# scrape all the years tracks - heavy handed, but easier than parsing track names and finding years from one big list

#
    
def save_track_year(params):
    name = params['name'].replace(' ', '').replace('/','%2F').lower()
    
    x = params['query'][0]
    y = params['query'][1]
    
    tracks_fname = "../v3_html_data/{}_{}_tracks.html.xz".format(name,params['year'])    
    tracks_url = "http://music.ishkur.com/musicbox.php?x={}&y={}&match=year".format(x,y)
    cache_url(tracks_url, tracks_fname)

In [33]:
process_with_progress(save_track_year, all_years)
print()

In [34]:
genre_first_year = defaultdict()

for entry in all_years:
    year = entry['year']
    name = entry['name']
    
    if name not in genre_first_year:
        genre_first_year[name] = year
    
    if genre_first_year[name] > year:
        genre_first_year[name] = year

In [75]:
#genre_first_year

In [ ]:
# for the network:

def clean_name(name):
    return name.replace(' ', '').replace('/','%2F').lower()

def year_start(name):
    return genre_first_year[name]

def self_node(name):
    return "{},{},{},".format(clean_name(name), clean_name(name), year_start(name))


print("source,target,start,end")
[print(self_node(name)+',\n'+clean_name(name)+',' )  for name in reversed(genre_to_scene.keys())]
print()

In [37]:
#

In [74]:
#all_years

In [39]:
fname = "../v3_html_data/{}_{}_tracks.html.xz".format("2-stepgarage", 1998)
with lzma.open(fname, 'rb') as f:            
    soup = BeautifulSoup(f.read(), "lxml")
    print()
    for source in soup.select('source'):
        parts = source['title'].split(' - ')
        print(parts)


['Groove Chronicles', 'Stone Cold']
['M-Dubs', 'Over Here (Sugar Shack Breakbeat Funk)']
['Antonio', 'Hyperfunk']
['Box Clever', 'Talk To Me']


In [68]:
#track_metadata = [(entry['scene'] = genre_to_scene[entry['name']]) for entry in all_years]

df = pd.DataFrame()

def make_tracks(entry):
    entry['scene'] = genre_to_scene[entry['name']]
    entry['slug'] = clean_name(entry['name'])
    
    tracks = []
    
    fname = "../v3_html_data/{}_{}_tracks.html.xz".format(entry['slug'], entry['year'])
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        
        for source in soup.select('source'):
            entry = entry.copy()
            parts = source['title'].split(' - ')
            entry['artist'] = parts[0]
            entry['title'] = parts[1]
            
            tracks.append(entry)
    
    return tracks
    
df = pd.DataFrame([track for entry in all_years for track in make_tracks(entry)], dtype=str)
df.rename({'name': 'genre'}, axis=1, inplace=True)
df['slug'] = df['genre'].apply(clean_name)

df = df[['slug', 'scene', 'genre', 'year', 'artist', 'title']]

In [69]:
df.sample(10)

,slug,scene,genre,year,artist,title
5623,germantrance,Eurotrance,German Trance,1994,Drax,Amphetamine
9929,synthpop,Eurodisco,Synthpop,1982,ABC,The Look Of Love
2784,detroittechno,Techno,Detroit Techno,2005,Keith Tucker,My Mental State
6768,indie,Chill Out,Indie,2012,Yppah,Happy To See You
1096,atmosphericjungle,Drum n Bass,Atmospheric Jungle,2012,Thesis,Blessing In Disguise
538,ambient,Ambient,Ambient,1984,Zero Kama,Azure-Lidded Woman (Pregnant Womb Of Non)
7506,miamibass,Bass,Miami Bass,1988,L'trimm,We Like The Cars That Go Boom
7501,miamibass,Bass,Miami Bass,1986,DJ KJ & MC Kooley C.,Let's Get This Party Started
9725,spacesynth,Eurodisco,Spacesynth,2001,Protonic Strom,Protonic Voices
3607,ebm,Industrial/Goth,EBM,1995,Snog,The Future


In [70]:
df.describe(include='all')

,slug,scene,genre,year,artist,title
count,11317,11317,11317,11317,11317,11317
unique,166,28,166,65,9594,10724
top,synthpop,House,Synthpop,2000,Sounds From The Ground,Move Your Body
freq,256,812,256,445,6,6


In [71]:
df.to_csv('../v3_tracks.csv',index=False)

In [44]:
# Descriptions:

In [46]:
#genre_to_scene.keys()

In [47]:
def get_description(genre):
    fname = "../v3_html_data/{}_description.html.xz".format(genre)
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        genre_name = soup.select('td.header', limit=1)[0].text
        genre_aka = soup.select('td.miniheader', limit=1)[0].text
        genre_scene = soup.select('td.subheader', limit=1)[0].text
        genre_emerge = soup.select('td.miniheader')[-1].text
    return {'name': genre_name,
           'aka':genre_aka,
           'scene':genre_scene,
           'emerged':genre_emerge}

In [72]:
df_genres = pd.DataFrame([get_description(clean_name(genre)) for genre in genre_to_scene.keys()])
df_genres['slug'] = df_genres['name'].apply(clean_name)
df_genres.rename({'name': 'genre'}, axis=1, inplace=True)
df_genres = df_genres[['slug', 'scene', 'genre', 'emerged', 'aka']]
df_genres.sample(10)

,slug,scene,genre,emerged,aka
103,electrohouse,House,Electrohouse,early 00s,"Electro House, Swedish House, Complextro"
51,braindance,Intelligent Dance Music,Braindance,mid 90s,IDM
56,soundtrack,Pioneers,Soundtrack,"early 50s, most likely","Theme music, background music, filler music, l..."
105,discohouse,House,Disco House,early 90s,"Funky House, Disco"
142,bleeptechno,Techno,Bleep Techno,late 80s,UK Techno
73,acidjazz,Downtempo,Acid Jazz,late 80s,"Jazzdance, Electro Swing, House Jazz, Funk Fusion"
154,rap,Hip Hop,Rap,mid to late 70s,"Freestyling, MCing, Hip Hop"
49,experimental,Pioneers,Experimental,who knows,Early electronic sounds
68,chillout,Chill Out,Chill Out,late 70s,"Ambient House, Balearic"
62,fm,Chiptune,FM,mid 80s,"Frequency Modulation, 16-bit, Yamaha, Genesis,..."


In [73]:
df_genres.to_csv('../v3_genres.csv',index=False)
#df_genres.sample(10)

In [ ]:
#
#for guide text descriptions:

def extract_desc(genre):
    fname = "../v3_html_data/{}_description.html.xz".format(genre)
    #print (fname)
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        desc = soup.select_one('td.td-copy').text
    return desc

scene = df_genres.iloc[0]['scene']

print("# {}".format(scene))
print()

for idx, r in df_genres.iterrows():
    
    if str(r['scene']) != scene:
        print("# {}".format(r['scene']))
        scene = r['scene']
    
    print()
    print("##", r['genre'])
    
    type_of = ""    
    if str(r['scene']) != 'nan':
        type_of = "in the " + r['scene'] + " scene,"
    
    if str(r['aka']) != 'nan':
        print("*aka: ", r['aka'].strip()+"*", type_of, 'emerged in', r['emerged'])
    else:
        print(type_of)

    fname = "../v3_html_data/{}_description.html.xz".format(r['slug'])
    with lzma.open(fname, 'rb') as f:            
        soup = BeautifulSoup(f.read(), "lxml")
        
        desc = extract_desc(r['slug'])

    print()
    print(desc)
    print()

In [66]:
#extract_desc('2-stepgarage')